## Introduction  {-}

In this project, the goal is to utilize the historical sales data for 45 Walmart stores, containing multiple departments, in different regions to predict future weekly sales for each department-store pair. I split this task to 10 2-month steps. That is, I start the 2-month prediction from 2011-03 using training data from a year earlier (i.e., 2010-02 to 2011-02). Then, I repeat continue this process every two months (referred to as a `fold`), i.e., predict 2011-05 and 2011-06 from data up to and including 2011-04 from 2010-02. I repeat this process 10 times, shown in details below:

- `fold 1`, predict 2011-03 to 2011-04 based on data from 2010-02 to 2011-02
- `fold 2`, predict 2011-05 to 2011-06 based on data from 2010-02 to 2011-04
- `fold 3`, predict 2011-07 to 2011-08 based on data from 2010-02 to 2011-06
- ...
- `fold 10`, predict 2012-09 to 2012-10 based on data from 2010-02 to 2012-08

For each `fold`, I evaluate the model predictions for expected weekly sales against the true weekly sales using the weighted mean absolute error (`WMAE`), giving a 5x multiplier for errors during holiday weeks. The goal is to obtain an average `WMAE` computed over the 10 `fold`s below $1580$.


## Pre-processing {-}

For pre-processing, I started with an exploration of the data in conjunction with the [variable description on Kaggle](https://www.kaggle.com/competitions/walmart-recruiting-store-sales-forecasting/data).

In this assignment, I am only using the training data, i.e., `train.csv`, (not the additional `features.csv`) which contains the following columns:

- `Store` - the store number
- `Dept` - the department number
- `Date` - the week
- `Weekly_Sales` -  sales for the given department in the given store
- `IsHoliday` - whether the week is a special holiday week

Since I am performing roolling 2-month predictions, for modeling each `fold`, I split the `train.csv` into a training dataset, i.e., period before the fold, and a test dataset, i.e., 2-month period during the fold.

Next, I went through the following pre-processing steps.


### Missing Values {-}

Looking at the data, as discussed on CampusWire (#766), the optimal approach to deal with missing values was to replace them with zeros. Doing so, similar to investigation by Prof. Liang, I got better performance when using a Random Forest Regression model in terms of average WMAE than other forms of interpolation, such as using values from the most recent week. As such, I decided to replace these missing values with 0.

Furthermore, I omitted store-dept pairs that did not exist in either training or test datasets and output `NaN` for the predicted weekly sales of those pairs that only exist in the test dataset.

### Feature Engineering {-}

After some investigations, I found that I can obtain better performance by extracting relevant information from the `Date` column. Specifically, I can convert `Date` to a week of the year (`Wk`) to ensure that for each week, I am predicting the sales for, I am using data from the same week as the year before. Similarly, I can extract a `Yr` variable from `Date` to make it easier to model the weekly sales as a function of week of the year and the year itself.

Through this project, I experimented with using additional features, such as `Yr2` (`Yr` squared) and `IsHoliday` (as a binary `0-1` variable) as well as converting features to categorical features. However, in the final model, I did not use any features besides `Store`, `Dept`, `Wk`, and `Yr`, with all being integer values, since they did not improve model performance.

### SVD Smoothing {-}

Similar to Prof. Liang's suggestion (#782), following the observation that data for the same department store across different stores appear similar, I decided to apply SVD smoothing to reduce the noise in the dataset. Specifically, this process involved gathering the data for the same department across all stores (`m` store) over all the available weeks (`n` weeks) to create the matrix $X_{m\times n}$. I only applied this smoothing when there were more rows, i.e., stores for the given department, than number of components chosen. Through experimentation, I found that `n_comps=20` yields the best model performance.

Since some stores could have missing values for some weeks, I filled in the missing values with 0 (similar to the [top scoring submission in the Kaggle competition](https://github.com/davidthaler/Walmart_competition_code/blob/master/grouped.forecast.R#L344)) prior to performing SVD.

Also, I subtracted the store mean from $X_{m\times n}$ prior to applying SVD. Then, to perform SVD smoothing, I chose the top `n_comps` components of the diagonal $D$ matrix in the SVD decomposition

$$
X_{m\times n}-\text{store.mean}=U_{m\times r}D_{r\times r}V_{r\times n}^t
$$

That is, I zeroed all elements on the diagonal in $D$ other than the top `ncomps` components to create $\tilde{D}_{r\times r}$ and multiplied it back to get the smoothed $X$, i.e., $\tilde{X}$ as follows:

$$
\tilde{X}=U_{m\times r}\tilde{D}_{r\times r}V_{r\times n}^t+\text{store.mean}
$$

Through this process, applied to all department if they meet the criteria for the minimum number of rows, I am intending to discard the noise data by selecting top principal components contributing the most to the data, i.e., signals.


## Implementation {-}

For my model implementation in Python, I experimented with a variety of models. Specifically, I started with a linear model (using `sklearn.linear_model.LinearRegression`), and then XGB-based regressor (`xgboost.XGBRegressor`). Finally, I found that a Random Forest Regressor (`sklearn.ensemble.RandomForestRegressor`) performs the best out of the three models by a large margin and it has a more reasonable runtime, esp. when compared to XGB-based regressor


### Tree-based model {-}

For the tree-based model, I used `sklearn.ensemble.RandomForestRegressor` with the following custom parameters, tuned through Grid Search:

- `n_jobs = -1`: This parameter specifies the number of jobs to run in parallel. I used `-1` to ensure that all processors are used for parallelization, which led to significant improvment in runtime.
- `n_estimators = 150`: This parameter specifies the number of trees in the forest. Using grid search, I found that 150 leads to the best model performance. Also, since the dataset used for training is relatively small, this value seemed reasonable for a random forest-based regressor
- `max_depth = 35`:  This parameter specifies the maximum depth a tree. A large `max_depth` would make the model more complex and increase the chances of overfitting to the training dataset.
- `max_samples = 0.9` (or `1.0` for `fold 5`): This parameters specifies the number of samples to draw from the training data ($X$) to train each base estimator. I found that using a smaller subset of data for all but `fold 5` led to better model performance. This is likely due to limited amount of training data available for predicting `fold 5` which has several corner cases and the necessity to use all the available data for training. For other folds, setting a value smaller than `1.0` helps add additional randomness to training which helps reduce the chance of overfitting to the training dataset.
- `min_samples_split = 2`: This parameter specifies the minimum number of samples required to split an internal node, which is a parameter affecting overfitting and underfitting. For this parameter, I found that using a smaller value helped with model performance as the model was less constrained and was able to learn more from the data, i.e., prevent underfitting.
- `min_samples_leaf = 1`: This parameter specifies the minimum number of samples required to be at a leaf node. Similar to `min_samples_split`, increasing the value of this parameter constrains the model and leads to underfitting. I found that using `1` led to the best performance and did not result in noticable overfitting.
- `random_state = $UID`: This parameter controls the randomness in model training, similar to a random seed. I specified by university ID to ensure reproducibility.

The rest of the parameters were left as default as tuning the above parameters led to the desired performance.

### Post-processing {-}

For `fold 5` (the last two months of the 2011), an additional post-processing step was taken to handle the special case of having a discrepancy in terms of the day of the week Christmas falls on in 2010 and 2011. Specifically, in 2010, most of the Christmas shopping sales occur in the week before the week of Christmas since Christmas day was on a Saturday (with weeks ending on Friday). In 2011, however, Christmas was on a Sunday, so there is one pre-Christmas shopping day for week 52. As such, I speculate that I can use a **"shift"** a fraction (close to $\frac{1}{\text{number days of week}}=\frac{1}{7}$) of sales predicted for week 51 in 2011 to week 52 to account for this discrepancy. However, looking at the predictions, for some store-dept pairs, it appeared that both weeks 51 and 52 had similar predicted weekly sales, where the bulge in sales in Christmas was less drastic. So, I limited the application of this shift to only those store-dept pairs whose week 51 predicted weekly sales are more than twice that of week 52.

Through my testing, I found that using a shift of $\frac{1}{12}$ actually led to most accurate results for this fold for `RandomForestRegressor` predictions. For store-dept pairs passing the application criteria, I found that scaling week 51 predicted weekly sales by $\frac{11}{12}$ and adding a $\frac{1}{12}$-th fraction of the original week 51 predicted weekly sales to that of week 52 resulted in the most accurate predictions for `fold 5`.

## Results {-}

Using the metric provided in the project description, for each fold, I evaluated my weekly sales predictions against the true weekly sales. Below, you can see the evaluation result for each model type. I also recorded the runtime of training and evaluating each model in seconds on each fold.

Per-fold results (WMAE and runtime) are shown below:

![WMAE and runtime per fold](2_perf_table.png "WMAE and runtime per fold")

Aggregate results are reported below:
- Total runtime over all 10 folds: $4\text{min }28.79\text{s}$
- Average runtime: $26.88s$
- Average WAME (weighted mean absolute error) over 10 folds: $1574.267$


*Runtime of training and evaluating each model in seconds on a local **device**, additional detail below.*

**Platform**

- MacBook Pro (15-inch, 2017)
  - *CPU:* 2.8 GHz Quad-Core Intel Core i7
  - *Memory:* 16 GB 2133 MHz LPDDR3
  - *GPU:* Radeon Pro 555 2 GB
  

## Discussion {-}

### Lessons Learned {-}

- Throughout this project, I realized the importance of handling noise in the dataset. Utilizing SVD smoothing, I was able to reduce the noise in the dataset and achieve significantly better regression performance. This demonstrated the need for in-depth analysis of data to help identify potential noise and ways of handling that noise.
- This project demonstrated the significant advantage and ease of use of ensemble tree-based models (i.e., Random Forest) when compared to linear ones when it comes to regression tasks, at least in Python. Specifically, through simple hyperparameter tuning and limited feature engineering, I was able to significantly beat the performance of linear regression and even baseline XBGRegressor. Linear regression in Python (`sklearn.linear_model.LinearRegression`) did not perform as well as R's `lm.fit` when it came to rank-deficient matrices, leading to significantly worse performance than tree-based models.
- Another important lesson was with regards to the importance of feature engineering and understanding the dataset. With the simple yet important realization (through data exploration) that sales for the same week of the year appear similar across years, I was able to engineer two features (`Wk` and `Yr`) that significantly boosted the model performance.

### Interesting Findings {-}

- Using `n_jobs=-1` resulted in significant runtime improvements during model training, on the order of `10x`.
- Using `IsHoliday` (binary `0-1`) as a feature resulted in significantly worse performance than otherwise (around +50\% `WMAE`). This highlighted the fact that more features do not necessarily lead to better performance.
- For `RandomForestRegressor`, I found that the squared `Yr` feature did not improve model performance despite the same feature improving performance for Linear Regression. This appears to align with the consensus that Random Forest regressor can model quadratic terms and relationships.